# ⚖️ Scales

Scales, provided by [`d3-scale-chromatic`](https://github.com/d3/d3-scale-chromatic)
provide an efficient way to use pre-calcluated values for a number color-based node and
link [shape](./Shapes.ipynb) properties.

Scales have a _domain_ (the expected values) and a _range_ (the colors to draw). Some
schemes (only) support _interpolation_ across their range, and other only offer discrete
values or even subschemes with different color bands.

Some experimentation may be needed to find an appropriate combination of these settings
for a given graph's data.

In [ ]:
if __name__ == "__main__" and "pyodide" in __import__("sys").modules:
    %pip install -q -r requirements.txt

In [ ]:
import ipywidgets as W
import traitlets as T
from ipyforcegraph import behaviors as B
from ipyforcegraph import graphs as G
from ipyforcegraph.behaviors import scales as SC

In [ ]:
fg = G.ForceGraph(layout=dict(flex="1", height="100%"))
controls = W.VBox(layout=dict(min_width="25em", flex="0"))
W.HBox(
    [controls, fg],
    layout=dict(
        min_height="400px", height="100%", flex="1", overflow="hidden", display="flex"
    ),
)

In [ ]:
n = 100
with fg.source.hold_trait_notifications():
    fg.source.nodes = [{"id": i, "value": i / n} for i in range(n)]
    fg.source.links = sum(
        [
            [
                {"id": i, "source": i, "target": i - 1 if i else n - 1},
                {"id": i, "source": i, "target": i % 5},
            ]
            for i in range(n)
        ],
        [],
    )

In [ ]:
ns = B.NodeShapes(color=SC.ColorScaleColumn("value", scheme=SC.Chromatic.viridis))
ls = B.LinkShapes(
    color=SC.ColorScaleColumn("id", domain=[0, n], scheme=SC.Chromatic.viridis)
)
fg.behaviors = [ns, ls]

In [ ]:
def make_a_scheme_picker(label, scale):
    scheme_opts = dict(options={v.name: v.value for v in SC.Chromatic})
    scheme = W.SelectionSlider(description=label, **scheme_opts)
    sub_scheme = W.Dropdown(
        description="subscheme", options={str(v): v for v in [None, *range(20)]}
    )
    interp = W.Checkbox(True, description="interpolate")
    T.link((scale, "interpolate"), (interp, "value"))
    T.link((scale, "scheme"), (scheme, "value"))
    T.link((scale, "sub_scheme"), (sub_scheme, "index"))
    T.dlink(
        (interp, "value"),
        (sub_scheme.layout, "visibility"),
        lambda x: "hidden" if x else "visible",
    )
    return W.VBox([scheme, interp, sub_scheme])

In [ ]:
controls.children = [
    make_a_scheme_picker("node", ns.color),
    make_a_scheme_picker("link", ls.color),
]